<a href="https://colab.research.google.com/github/deguc/datascience/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


#マルチレイヤーネットワーク

def DateSets(y=[0,1,1,1],N=50,loc=0,scale=0.05):

  #N=10         データのサイズ
  #loc          平均
  #scale=0.05   標準偏差

  X_=np.array([[0,0],[0,1],[1,0],[1,1]])#学習用データ

  #元のデータにノイズを足す。
  X0=X_[0]+np.random.normal(size=(N,2),loc=loc,scale=scale)
  X1=X_[1]+np.random.normal(size=(N,2),loc=loc,scale=scale)
  X2=X_[2]+np.random.normal(size=(N,2),loc=loc,scale=scale)
  X3=X_[3]+np.random.normal(size=(N,2),loc=loc,scale=scale)
  X=np.concatenate((X0,X1,X2,X3))   #データを結合する。

  #正解ラベル
  y0=np.full(N,y[0])
  y1=np.full(N,y[1])
  y2=np.full(N,y[2])
  y3=np.full(N,y[3])
  y=np.concatenate((y0,y1,y2,y3))   #データを結合する

  return X,y

def plot(X,y,c=['red','blue','green','orange']):
  #データをプロットする。
  n=y.max()
  for k in range(n+1):
    plt.scatter(X[y==k,0],X[y==k,1],c=c[k]);

def decision_regions(x,y,model):

  #平面を分割する。

  #カラーマップの準備
  colors=['red','blue','green','orange']
  cmap=ListedColormap(colors[:len(np.unique(y))])

  #グリッドの作成
  x1=np.arange(-0.2,1.2,0.02)   #横軸
  x2=np.arange(-0.2,1.2,0.02)   #縦軸
  xx1,xx2=np.meshgrid(x1,x2)    #メッシュ
  z=np.array([xx1.ravel(),xx2.ravel()]).T   #座標点の作成

  Z=model(z).reshape(xx1.shape)   #各座標点を分類する

  plot(x,y,c=colors) #データをプロットする。

  plt.contourf(xx1,xx2,Z,alpha=0.2,cmap=cmap) #平面の分割
  plt.show()

def onehot(X):
  return np.identity(len(X))[y]

def sigmoid(x):
  return 1/(1+np.exp(-x)+1e-7)

class Sigmoid:
  def __init__(self):

    self.y=None

  def forward(self,X):
    y=sigmoid(X)
    self.y=y
    return y

  def backward(self,delta):
    y=self.y
    return delta*(1-y)*y




class Affine: #アフィンレイア
  def __init__(self,input_dim,output_dim):
    self.W=np.random.randn(input_dim,output_dim)  #重み
    self.b=np.zeros(output_dim) #バイアス

    self.dW=None  #重みの勾配
    self.db=None  #バイアスの勾配

    self.X=None  #入力データを保存

  def forward(self,X):  #順伝播

    self.X=X    #入力データを保存

    return np.matmul(X,self.W)+self.b

  def backward(self,delta):#逆伝播

    X=self.X
    W=self.W

    self.dW=np.matmul(X.T,delta)
    self.db=np.sum(delta,axis=0)

    return np.matmul(delta,W.T)

  def update(self,eta=0.1):  #重みバイアスの更新　SGD

    self.W -= eta*self.dW
    self.b -= eta*self.db

    return None


class Twin: #２層で構成されたニューロネットワーク
  def __init__(self,input_dim,hidden_dim,output_dim):

    self.l1=Affine(input_dim,hidden_dim)
    self.a1=Sigmoid()  #活性化関数
    self.l2=Affine(hidden_dim,output_dim)

  def forward(self,X):  #順伝播

    y=self.l1.forward(X)
    y=self.a1.forward(y)
    y=self.l2.forward(y)

    return y

  def backward(self,delta): #逆伝播
    delta=self.l2.backward(delta)
    delta=self.a1.backward(delta)
    delta=self.l1.backward(delta)

    return delta

  def update(self):  #パラメータの更新
    self.l1.update()
    self.l2.update()


X,y=DateSets(y=[0,1,1,1],N=5) #学習用のデータの準備
input_dim=X.shape[1]
output_dim=np.max(y)+1
model=Twin(input_dim,5,output_dim)
y=model.forward(X)
print(y)


[[ 0.72267521  0.89488957]
 [ 0.76135405  0.91977235]
 [ 0.653418    0.82752969]
 [ 0.70504918  0.89451953]
 [ 0.59654398  0.77749662]
 [-0.17216104  0.12702011]
 [-0.10215442  0.20347779]
 [-0.09950249  0.20354599]
 [ 0.02882341  0.33905717]
 [-0.16451544  0.1361038 ]
 [-0.00492883  0.13365894]
 [-0.01618956  0.1282983 ]
 [ 0.02807238  0.16336494]
 [-0.025229    0.11966667]
 [-0.05322884  0.09561471]
 [-0.46195903 -0.18276034]
 [-0.47343404 -0.19123761]
 [-0.50331188 -0.21081394]
 [-0.48760532 -0.20183013]
 [-0.49065933 -0.20380093]]
